# Azure ML - Environments

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Environmnets

Hemos visto cómo crear `Compute Instance`s y `Compute Cluster`s, pero a la hora de entrenar un modelo necesitamos tener ciertas librerías instaladas. Para ello podemos crear entornos. La forma de hacerlo en Azure ML es mediante imágenes `Docker` o entornos de `Conda`.

Para mejorar la portabilidad normalemente se usan entornos `Conda` dentro de entornos `Docker`.

![Azure ML Environments](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/azureml-environment.webp)

#### Crear un `Environment` desde la interfaz gráfica

Para crear un nuevo `ENvironment` nos vamos a nuestro workspace y en el menú de la izquierda seleccionamos `Environments`, dentro de `Assets`. Vamos a la pestaña `Custom envronments` y le damos a `Create`, a partir de aquí

 * Seleccionamos un nombre, en mi caso le voy a poner `environment-GUI`
 * En `Select environment source` seleccionamos `Use existing docker image with conda file`
 * En `Container registry image path` introducimos `pytorch/pytorch:2.5.0-cuda12.4-cudnn9-devel`

Pulsamos en `Next`. Ahora tenemos que escribir el archivo `conda.yml` que queremos usar. En mi caso voy a usar el siguiente

```yaml
name: transformers
channels:
  - conda-forge
dependencies:
  - python=3.11
  - pip
  - pip:
    - transformers
    - accelerate
    - datasets
```

Pulsamos en `Next` y en `Create` y ya tenemos nuestro entorno creado

### Crear un `Environment` con el CLI de Azure ML

Para crear un entorno, primero creamos el archivo de configuración del entorno de conda

In [6]:
!echo "name: transformers"  > conda_env.yml
!echo "channels:"          >> conda_env.yml
!echo "  - conda-forge"    >> conda_env.yml
!echo "dependencies:"      >> conda_env.yml
!echo "  - python=3.11"    >> conda_env.yml
!echo "  - pip"            >> conda_env.yml
!echo "  - pip:"           >> conda_env.yml
!echo "    - transformers" >> conda_env.yml
!echo "    - accelerate"   >> conda_env.yml
!echo "    - datasets"     >> conda_env.yml

Ahora creamos el entorno con el siguiente comando

In [14]:
!GROUP='rg-azure-ml' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="environment-CLI" && \
CONDA_FILE="conda_env.yml" && \
DOCKER_IMAGE="pytorch/pytorch:2.5.0-cuda12.4-cudnn9-devel" && \
az ml environment create --name $NAME --resource-group $GROUP --workspace-name $WORKSPACE --conda-file $CONDA_FILE --image $DOCKER_IMAGE

{
  "conda_file": {
    "channels": [
      "conda-forge"
    ],
    "dependencies": [
      "python=3.11",
      "pip",
      {
        "pip": [
          "transformers",
          "accelerate",
          "datasets"
        ]
      }
    ],
    "name": "transformers"
  },
  "creation_context": {
    "created_at": "2024-10-28T20:00:55.768966+00:00",
    "created_by": "Maximo Fernandez Nuñez",
    "created_by_type": "User",
    "last_modified_at": "2024-10-28T20:00:55.768966+00:00",
    "last_modified_by": "Maximo Fernandez Nuñez",
    "last_modified_by_type": "User"
  },
  "id": "azureml:/subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-CLI/environments/environment-CLI/versions/1",
  "image": "pytorch/pytorch:2.5.0-cuda12.4-cudnn9-devel",
  "name": "environment-CLI",
  "os_type": "linux",
  "resourceGroup": "rg-azure-ml",
  "tags": {},
  "version": "1"
}


### Crear un `Environment` con el SDK de Python

Para crear un entorno con el SDK de Python hacemos lo siguiente

In [3]:
from azure.ai.ml.entities import Environment

docker_image = "pytorch/pytorch:2.5.0-cuda12.4-cudnn9-devel"
conda_file = "conda_env.yml"
name = "environment-python"

env_docker_conda = Environment(
    image=docker_image,
    conda_file=conda_file,
    name=name,
    description="Environment created from a Docker image plus Conda environment. Pyhton SDK",
)

result = ml_client.environments.create_or_update(env_docker_conda)

In [4]:
result

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'pytorch/pytorch:2.5.0-cuda12.4-cudnn9-devel', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'environment-python', 'description': 'Environment created from a Docker image plus Conda environment. Pyhton SDK', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-Python-SDK/environments/environment-python/versions/1', 'Resource__source_path': '', 'base_path': '/home/wallabot/Documentos/web/portafolio/posts', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f00ba102650>, 'serialize': <msrest.serialization.Serializer object at 0x7f00ba113010>, 'version': '1', 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['p